# Note 1. Setting non-uniform weights. 
When using biasin techniques such as umbrella sampling or metadynamics, the initial weights $w_0$ are not uniform. It is possible to specify $w_0$ when initializing the reweighting class. Below we create random weights, just to make an example. 

In [1]:
import os,sys
import numpy as np
#this is a trick to get the parent directory 
directory_to_bme = os.getcwd()[:-8]
sys.path.append(directory_to_bme)
import bme_reweight as bme

# generate random weights 
w0 = np.random.random(20000)
# normalize
w0 /= np.sum(w0)

# initialize reweighting class with weights                                                                                                                                
rew = bme.Reweight(w0=list(w0))


# Set non-uniform initial weights from file. Sum= 1.0 20000


# Note 3. Using inequality restraints

There might be situations in which the experimental data is know as upper or lower boundaries. For noisy NOE data, we can for example say that a given proton proton distance cannot be more than e.g. 0.6nm, resulting in an upper boundary.  In some other cases, it is possible to assign unobserved NOE (see [here](http://advances.sciencemag.org/content/4/5/eaar8521) ), resulting in lower boundaries instead.

Both types of inequality restraints can be used in BME. To do so, it is sufficient to *flag* the experimental datafile as shown below:

In [2]:
%cat ../data/uNOE_exp.dat | head


# DATA=NOE PRIOR=GAUSS  
C1_H1'_C2_H4' 5.2 0.1 LOWER
C1_H1'_C3_1H2' 5.2 0.1 LOWER
C1_H1'_C3_H3' 5.2 0.1 LOWER
C1_H1'_C3_H4' 5.2 0.1 LOWER
C1_H1'_C3_2H5' 5.2 0.1 LOWER
C1_H1'_C4_H3' 5.2 0.1 LOWER
C1_H1'_C4_H4' 5.2 0.1 LOWER
C1_H1'_C4_2H5' 5.2 0.1 LOWER
C1_1H2'_C3_1H2' 4.6 0.1 LOWER


As usual, the first line is the header, and the experimental data starts from line 2. The boundary is the second column, the third is the error and the fourth column can be either `UPPER` or `LOWER`. The calculation of NOE from simulation is identical to the case of standard NOE restraints.


In [3]:

# define name of files                                                                               
exp_inequality = '../data/uNOE_exp.dat'
calc_inequality = '../data/uNOE_calc.dat'

# initialize reweighting class                                                                                                                                
rew = bme.Reweight()

# load data                                                                                      
rew.load(exp_inequality,calc_inequality)
# do optimization using theta=2                                                                                                                               
chi2_before,chi2_after, srel = rew.optimize(theta=2)

print "# CHI2 before minimization:     %8.4f" % (chi2_before)
print "# CHI2 after minimization:      %8.4f" % (chi2_after)
print "# Fraction of effective frames: %8.4f" % (np.exp(srel))

# Warning: expt average        C1_H5_C2_2H5'=1.0555e-04 is larger than maximum value in simulation 1.0435e-04
# Warning: expt average         C3_H5_C3_H4'=1.0555e-04 is larger than maximum value in simulation 1.9996e-05
# Warning: expt average         C2_H5_C2_H4'=2.4414e-04 is larger than maximum value in simulation 2.2224e-05
# Warning: expt average         C2_H5_C3_H3'=2.4414e-04 is larger than maximum value in simulation 7.0152e-05
# Warning: expt average        C2_H5_C3_1H5'=2.4414e-04 is larger than maximum value in simulation 1.7909e-04
# Warning: expt average        C3_H1'_C1_H3'=2.4414e-04 is larger than maximum value in simulation 1.0643e-04
# Warning: expt average         C3_H6_C4_H1'=2.4414e-04 is larger than maximum value in simulation 1.6042e-04
# Warning: expt average       C1_H3'_C3_2H5'=7.7431e-04 is larger than maximum value in simulation 2.6977e-04
# Warning: expt average         C1_H5_C2_H4'=7.7431e-04 is larger than maximum value in simulation 2.1913e-04
# Warning:

We can now check if the inequality restraints have been fulfilled. The easiest way is to call the function `weight_exp()` and check the output

In [4]:
rew.weight_exp(exp_inequality,calc_inequality,"inequality_test")

(4.32028283304481, 5.153855946339172e-06)

In [5]:
%cat inequality_test.stats.dat


# uNOE_exp.dat vs. uNOE_calc.dat srel=-8.1719e-01
#  Label             avg exp sigma exp avg before avg after 
   C1_H1'_C2_H4'   5.058e-05 5.836e-06 7.065e-06 1.400e-05  5.200e+00 1.000e-01 7.219e+00 6.441e+00 
   C1_H1'_C3_1H2'  5.058e-05 5.836e-06 4.675e-05 5.058e-05  5.200e+00 1.000e-01 5.269e+00 5.200e+00 
   C1_H1'_C3_H3'   5.058e-05 5.836e-06 7.155e-06 4.764e-06  5.200e+00 1.000e-01 7.204e+00 7.709e+00 
   C1_H1'_C3_H4'   5.058e-05 5.836e-06 2.534e-06 4.503e-06  5.200e+00 1.000e-01 8.564e+00 7.782e+00 
   C1_H1'_C3_2H5'  5.058e-05 5.836e-06 1.931e-06 2.612e-06  5.200e+00 1.000e-01 8.961e+00 8.521e+00 
   C1_H1'_C4_H3'   5.058e-05 5.836e-06 7.262e-06 1.512e-06  5.200e+00 1.000e-01 7.186e+00 9.334e+00 
   C1_H1'_C4_H4'   5.058e-05 5.836e-06 1.722e-06 1.184e-06  5.200e+00 1.000e-01 9.134e+00 9.722e+00 
   C1_H1'_C4_2H5'  5.058e-05 5.836e-06 1.286e-06 6.472e-07  5.200e+00 1.000e-01 9.589e+00 1.075e+01 
   C1_1H2'_C3_1H2' 1.055e-04 1.377e-05 1.619e-05 1.283e-05  4.600e+00 1.000e-01 6

We can check how many and which violations are present before/after reweighting

In [6]:
print "Violations in the original simulation"

for line in open("inequality_test.stats.dat"):
    if("#" in line): continue
    label = line.split()[0]
    exp = float(line.split()[5])
    old_avg = float(line.split()[7])
    new_avg = float(line.split()[8])
    if(old_avg<exp):
        print " %20s Original average: %5.3f, new average %5.3f experimental boundary: %5.3f" % (label,old_avg,new_avg,exp)

print "Violations with optimized weights"

for line in open("inequality_test.stats.dat"):
    if("#" in line): continue
    label = line.split()[0]
    exp = float(line.split()[5])
    old_avg = float(line.split()[7])
    new_avg = float(line.split()[8])
    if(new_avg< exp):
        print " %20s Original average: %5.3f, new average %5.3f experimental boundary: %5.3f" % (label,old_avg,new_avg,exp)
    

Violations in the original simulation
        C1_H5_C4_1H2' Original average: 3.032, new average 4.599 experimental boundary: 4.600
         C1_H5_C4_H3' Original average: 3.441, new average 4.600 experimental boundary: 4.600
       C2_H1'_C4_1H2' Original average: 4.951, new average 5.200 experimental boundary: 5.200
       C2_H3'_C1_2H5' Original average: 3.986, new average 4.862 experimental boundary: 4.600
      C2_1H5'_C1_2H5' Original average: 4.206, new average 5.145 experimental boundary: 4.600
      C2_2H5'_C1_2H5' Original average: 2.610, new average 4.597 experimental boundary: 4.600
      C4_1H2'_C1_2H5' Original average: 4.424, new average 5.778 experimental boundary: 4.600
        C1_H5_C3_1H2' Original average: 2.994, new average 4.334 experimental boundary: 4.000
       C4_H3'_C3_2H5' Original average: 3.246, new average 3.999 experimental boundary: 4.000
         C1_H5_C3_H3' Original average: 2.619, new average 3.515 experimental boundary: 3.300
        C1_H6_C4_1H2' 

# Note 4. Using multiple sets of data at the same time
The BME  code allows the possibility to restrain the simulation using multiple sources of data at the same time. For example, we here use NOE, J couplings and unobserved NOEs at the same time by loading all data using the `load()` function. 



In [7]:
exp_inequality = '../data/uNOE_exp.dat'
calc_inequality = '../data/uNOE_calc.dat'
exp_couplings = '../data/couplings_exp.dat'
calc_couplings = '../data/couplings_calc.dat'
exp_noe = '../data/noe_exp.dat'
calc_noe = '../data/noe_calc.dat'

# initialize reweighting class                                                                                                                                
rew = bme.Reweight()

# load inequality constraints                                                                                
rew.load(exp_inequality,calc_inequality)
# load j-couplings
rew.load(exp_couplings,calc_couplings)
# load NOE
rew.load(exp_noe,calc_noe)

# do optimization using theta=2                                                                                                                               
chi2_before,chi2_after, srel = rew.optimize(theta=2)

print "# CHI2 before minimization:     %8.4f" % (chi2_before)
print "# CHI2 after minimization:      %8.4f" % (chi2_after)
print "# Fraction of effective frames: %8.4f" % (np.exp(srel))

# Warning: expt average        C1_H5_C2_2H5'=1.0555e-04 is larger than maximum value in simulation 1.0435e-04
# Warning: expt average         C3_H5_C3_H4'=1.0555e-04 is larger than maximum value in simulation 1.9996e-05
# Warning: expt average         C2_H5_C2_H4'=2.4414e-04 is larger than maximum value in simulation 2.2224e-05
# Warning: expt average         C2_H5_C3_H3'=2.4414e-04 is larger than maximum value in simulation 7.0152e-05
# Warning: expt average        C2_H5_C3_1H5'=2.4414e-04 is larger than maximum value in simulation 1.7909e-04
# Warning: expt average        C3_H1'_C1_H3'=2.4414e-04 is larger than maximum value in simulation 1.0643e-04
# Warning: expt average         C3_H6_C4_H1'=2.4414e-04 is larger than maximum value in simulation 1.6042e-04
# Warning: expt average       C1_H3'_C3_2H5'=7.7431e-04 is larger than maximum value in simulation 2.6977e-04
# Warning: expt average         C1_H5_C2_H4'=7.7431e-04 is larger than maximum value in simulation 2.1913e-04
# Warning:

Note that the $\chi^2$ considers all data collectively. If we want to calculate the new averages for the different types of data we can call the `weight_exp()` function


In [8]:
# load inequality constraints                                                                                
before, after = rew.weight_exp(exp_inequality,calc_inequality,"note2_uenoe")
print "# CHI2 uNOE before minimization:     %8.4f" % (before)
print "# CHI2 uNOE after minimization:      %8.4f" % (after)

# load j-couplings
before, after = rew.weight_exp(exp_couplings,calc_couplings,"note2_couplings")
print "# CHI2 couplings before minimization:%8.4f" % (before)
print "# CHI2 couplings after minimization: %8.4f" % (after)

# load NOE
before, after = rew.weight_exp(exp_noe,calc_noe,"note2_noe")
print "# CHI2 NOE before minimization:      %8.4f" % (before)
print "# CHI2 NOE after minimization:       %8.4f" % (after)




# CHI2 uNOE before minimization:       4.3203
# CHI2 uNOE after minimization:        0.0000
# CHI2 couplings before minimization:  1.1525
# CHI2 couplings after minimization:   0.1965
# CHI2 NOE before minimization:        3.1469
# CHI2 NOE after minimization:         0.0467
